In [ ]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Model, load_model
from keras.layers import *
from keras import backend as K
from keras import optimizers, callbacks, regularizers
import numpy as np
import pandas as pd
import cv2, h5py

In [ ]:
batch_size = 2**6
print("Batch size:", batch_size)

## Model (InceptionResnet V2)

In [ ]:
#"""
train_datagen = ImageDataGenerator(rotation_range=1,
        width_shift_range=0.1,
        height_shift_range=0.1,
        rescale=1./255,
        shear_range=0.05,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(
        'data2/train',
        target_size=(299, 299),
        batch_size=batch_size,
        class_mode='categorical')
#"""

In [ ]:
#"""
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(
        #'data2/train',
        'train_aug',
        target_size=(299, 299),
        batch_size=batch_size,
        class_mode='categorical')
#"""

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
        'data2/validation',
        target_size=(299, 299),
        batch_size=batch_size,
        class_mode='categorical')

In [ ]:
base_model = InceptionResNetV2(weights='imagenet', include_top=False)

In [ ]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.00))(x)
x = Dropout(0.3)(x)
predictions = Dense(128, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

In [ ]:
adadelta = optimizers.Adadelta(lr=1.2, rho=0.95, epsilon=None, decay=0.1)
model.compile(loss='categorical_crossentropy',
              optimizer=adadelta,
              metrics=['acc'])

### Training

In [ ]:
tensorboard = callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=16, write_grads=True , write_graph=True)
model_checkpoints = callbacks.ModelCheckpoint("inceptionresnet-{val_loss:.3f}-{val_acc:.3f}.h5", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=0)

In [ ]:
!rm -R logs

In [ ]:
!nvidia-settings -a [gpu:0]/GPUFanControlState=1
!nvidia-settings -a [fan:0]/GPUTargetFanSpeed=100

In [ ]:
print("Training Progress:")
model_log = model.fit_generator(train_generator, validation_data=validation_generator,
          epochs=4, workers=5, use_multiprocessing=True,
          callbacks=[tensorboard, model_checkpoints])

In [ ]:
pd.DataFrame(model_log.history).to_csv("inception-resnet-history.csv")

### Fine Tuning
* 632 block8_1
* 648 block8_2
* 664 block8_3
* 680 block8_4
* 696 block8_5
* 712 block8_6

In [ ]:
#model.load_weights("inceptionresnet-1.277-0.630.h5")

In [ ]:
for layer in base_model.layers[:712]:
    layer.trainable = False
for layer in base_model.layers[712:]:
    layer.trainable = True

In [ ]:
from keras.optimizers import SGD
adadelta = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.1)
model.compile(optimizer=adadelta, loss='categorical_crossentropy',metrics=['acc'])

print("Training Progress:")
model_log = model.fit_generator(train_generator, validation_data=validation_generator,
          epochs=4,
          callbacks=[tensorboard, model_checkpoints])

In [ ]:
for layer in base_model.layers[:632]:
    layer.trainable = False
for layer in base_model.layers[632:]:
    layer.trainable = True

In [ ]:
from keras.optimizers import SGD
adadelta = optimizers.Adadelta(lr=1.2, rho=0.95, epsilon=None, decay=0.2)
model.compile(optimizer=adadelta, loss='categorical_crossentropy',metrics=['acc'])

print("Training Progress:")
model_log = model.fit_generator(train_generator, validation_data=validation_generator,
          epochs=20,
          callbacks=[tensorboard, model_checkpoints])

## Evaluation

In [ ]:
from keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
%config InlineBackend.figure_format = 'retina'
import itertools, pickle

from glob import glob
class_names = glob("train_aug/*") # Reads all the folders in which images are present
class_names = sorted(class_names) # Sorting them
fixed_classes = []
for class_name in class_names:
    fixed_classes.append(class_name[10:])
name_id_map = dict(zip(range(len(class_names)), fixed_classes))
og_classes = [str(x) for x in range(1,129)]

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    'data2/validation', shuffle=False,
    target_size=(299, 299),
    batch_size=batch_size,
    class_mode='categorical')

In [ ]:
Y_pred = model.predict_generator(validation_generator, 6322 // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)

In [ ]:
corr_preds = []
for pred in y_pred:
    corr_preds.append(int(name_id_map[pred]))

In [ ]:
print('Classification Report')
print(classification_report(validation_generator.classes, y_pred, target_names=og_classes))